In [ ]:
import os
import numpy as np
import tensorflow as tf

from show_graph import *

In [ ]:
tf.reset_default_graph()

c = tf.constant(5., name='c')
y = tf.constant(10., name='y')
v = tf.Variable(0., name='v')

r = c * v
loss = tf.square(r - y, 'loss')

optimizer = tf.train.GradientDescentOptimizer(0.01)
step = optimizer.minimize(loss, name='minimize')

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  print('before:', sess.run(r))
  for _ in range(20):
    sess.run(step)
  print('after:', sess.run(r))

show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
tf.reset_default_graph()

c = tf.constant(5., name='c')
y = tf.constant(10., name='y')
v = tf.Variable(0., name='v')

r = c * v
loss = tf.square(r - y, 'loss')

optimizer = tf.train.GradientDescentOptimizer(0.02)
gradients = optimizer.compute_gradients(loss)
update_op = optimizer.apply_gradients(gradients, name='update_op')

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  print('before:', sess.run(r))
  for _ in range(20):
    sess.run(update_op)
  print('after:', sess.run(r))

show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
tf.reset_default_graph()

c = tf.constant(5., name='c')
y = tf.constant(10., name='y')
v = tf.Variable(0., name='v')
g = tf.Variable(0., trainable=False, name='g')

r = c * v
loss = tf.square(r - y, 'loss')

optimizer = tf.train.GradientDescentOptimizer(0.002)
gradients = optimizer.compute_gradients(loss)

reset_op = tf.assign(g, 0.)
acc_op = tf.assign_add(g, gradients[0][0])

update_op = optimizer.apply_gradients([(g,v)])

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  print('before:', sess.run(r))
  for i in range(40):
    sess.run(acc_op)
    print('{} step: {}'.format(i, sess.run([r, g])))
    if i % 2 == 0:
        sess.run(update_op)
        sess.run(reset_op)
    
  print('after:', sess.run(r))

show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
tf.reset_default_graph()

c = tf.constant(5., name='c')
y = tf.constant(10., name='y')
v = tf.Variable(0., name='v')
acc_grads = tf.Variable(0., trainable=False, name='acc_grads')
counter = tf.Variable(0, trainable=False, name='counter')

r = c * v
loss = tf.square(r - y, 'loss')

optimizer = tf.train.GradientDescentOptimizer(0.002)
gradients = optimizer.compute_gradients(loss)

acc_op = tf.assign_add(acc_grads, gradients[0][0])
apply_op = optimizer.apply_gradients([(acc_grads,v)])
with tf.control_dependencies([apply_op]):
  update_op = tf.assign(acc_grads, 0.)

reset_counter_op = tf.assign(counter, 0)
inc_counter_op = tf.assign_add(counter, 1)

step = tf.cond(tf.greater(counter, 1),
               lambda: tf.group(acc_op, inc_counter_op),
               lambda: tf.group(update_op, reset_counter_op))

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  print('before:', sess.run(r))
  for i in range(40):
    print('{} step: {}'.format(i, sess.run([r, acc_grads, counter])))
    sess.run(step)
    
  print('after:', sess.run(r))
